In [10]:
import numpy as np
import random

def vary_groups(group, n_el, options2choose, idx=0, random_array=np.array([], dtype='int32')):
    """
    Group sections and materials of the elements
    that are in the same group.
    """
    random_array_provided = False
    if random_array.shape[0] > 0:
        random_array_provided = True
            
    if group.shape[0] > 1:
        arr = np.zeros((n_el))
        group_array = np.ones((group.shape[0]))
        
        for g in range(group.shape[0]):
            if random_array_provided:
                selected = options2choose[random_array[g]][idx]
            else:
                random_array = np.concatenate((random_array, [random.randint(0, options2choose.shape[0]-1)]))
                random_array = random_array.astype('int32')
                selected = options2choose[random_array[-1]][idx]

            group_array[g] = selected
            for element in group[g]:
                if element <= 0:
                    break
                arr[element-1] = selected
    else:
        if random_array_provided:
            selected = options2choose[random_array[-1]][idx]
        else:
            random_array = np.array([random.randint(0, options2choose.shape[0]-1)], dtype='int32')
            selected = options2choose[random_array[-1]][idx]
        group_array = np.array([selected])
        arr = np.copy(group_array)
        

    return group_array, arr, random_array


def group_results(Fs, group):
    """
    Group F and sigma of the elements
    that are in the same group.
    """
    if group.shape[0] > 1:
        group_array = np.zeros((group.shape[0]))
        for g in range(group.shape[0]):
            for element in group[g]:
                if element <= 0:
                    break
                if abs(Fs[element-1]) > abs(group_array[g]):
                    group_array[g] = Fs[element-1]
    else:
        group_array = np.array([np.max(np.abs(Fs))], dtype='float64')
    
    return group_array

In [11]:
import numpy as np
from numpy.linalg import inv

def FEM(x, y, conec, prop_group, secoes, material, forcas, GDL_rest):
    """
    Calcula a resposta estrutural da estrutura.

    Parameters:
            x (np.float array of shape (n_nos,)):
                Coordinates x of nodes
            y (np.float array of shape (n_nos,)):
                Coordinates y of nodes
            conec (np.int array of shape (n_el, 4)): 
                [elemento, grupo,  nó_1,  nó_2]
                
            prop_group (np.int array of shape (n_group, 2)):
                Seção e material adotados em cada grupo
                
            secoes (np.float array of shape (n_sec, 13)):
                [Área, b, t, Ix, Iy, rx, ry, rz_min, wdt, J, W, x, s4g]
            material (np.float array of shape (n_mat, 3)):
                [Young_modulus, fy_k, density]
                
            forcas (np.float array of shape (n_forcas, 3)):
                [nó, Fx, Fy]
                
            GDL_rest (np.int array of shape (n_rest, 3)):
                [nó, rest_x, rest_y]
            
    Returns:
            desloc (np.float array of shape (n_nos*2,)):
                displacements of nodes
            fn (np.float array of shape(n_el,)):
                axial force in the elements
            ten (np.float array of shape(n_el,))
                tension in the elements
            reacoes (np.float array of shape (n_nos*2,)):
                reactions on the nodes
    """

    no = np.arange(1, len(x)+1)  # numeração dos nós
    n_nos = no[-1]  # número de nós
    n_el = conec.shape[0]
    n_forcas = forcas.shape[0]
    n_rest = GDL_rest.shape[0]  # número de nós restringidos

    # Cálculo da estrutura
    GDL = 2*n_nos  # graus de liberdade da estrutura
    K = np.zeros((GDL, GDL))  # matriz rigidez global

    # Cálculo da matriz de cada elemento
    for el in range(n_el):
        # Comprimento do elemento "el"
        no1 = conec[el][-2]-1
        no2 = conec[el][-1]-1

        L = np.sqrt((x[no2] - x[no1])**2 + (y[no2] - y[no1])**2)

        # Propriedades
        s = prop_group[conec[el][1]-1][0]
        m = prop_group[conec[el][1]-1][1]
        A = secoes[s-1][0]
        E = material[m-1][0]

        # Cossenos diretores a partir das coordenadas dos nós do elemento
        cs = (x[no2] - x[no1])/L  # cosseno
        sn = (y[no2] - y[no1])/L  # seno

        # Matriz de rigidez do elemento "el"
        k = E*A/L
        ke = np.array([
            [k, -k],
            [-k, k]
        ])

        T = np.array([
            [cs, sn, 0, 0],
            [0, 0, cs, sn]
        ])

        kg = T.transpose() @ ke @ T

        # Inserção na matriz de rigidez global
        for i in range(2):  # superposição da sub-matriz (1-2,1-2) da matriz elementar
            ig = (no1)*2+i
            for j in range(2):
                jg = (no1)*2+j
                K[ig][jg] = K[ig][jg] + kg[i][j]

        for i in range(2):  # superposição da sub-matriz (3-4,3-4) da matriz elementar
            ig = (no2)*2+i
            for j in range(2):
                jg = (no2)*2+j
                K[ig][jg] = K[ig][jg] + kg[i+2][j+2]

        for i in range(2):  # superposição das sub-matrizes (1-2,3-4) e ((3-4,1-2) da matriz elementar
            ig = (no1)*2+i
            for j in range(2):
                jg = (no2)*2+j
                K[ig][jg] = K[ig][jg] + kg[i][j+2]
                K[jg][ig] = K[jg][ig] + kg[j+2][i]

    # Vetor de forças Global
    F = np.zeros((GDL, 1))
    for i in range(n_forcas):
        F[2*int(forcas[i][0])-2] = forcas[i][1]
        F[2*int(forcas[i][0])-1] = forcas[i][2]

    # guardamos os originais de K e F
    Kg = np.copy(K)
    Fg = np.copy(F)

    # Aplicar Restrições (condições de contorno)
    for k in range(n_rest):
        # Verifica se há restrição na direção x
        if GDL_rest[k][1] == 1:
            j = 2*GDL_rest[k][0]-2

            # Modificar Matriz de Rigidez
            for i in range(GDL):
                Kg[j][i] = 0  # zera linha
                Kg[i][j] = 0  # zera coluna

            Kg[j][j] = 1     # valor unitário na diagonal principal
            Fg[j] = 0

        # Verifica se há restrição na direção y
        if GDL_rest[k][2] == 1:
            j = 2*GDL_rest[k][0]-1

            # Modificar Matriz de Rigidez
            for i in range(GDL):
                Kg[j][i] = 0   # zera linha
                Kg[i][j] = 0   # zera coluna

            Kg[j][j] = 1     # valor unitário na diagonal principal
            Fg[j] = 0

    # Cálculo dos deslocamentos
    desloc = inv(Kg) @ Fg  # Vetor dos deslocamentos
    # print(f'Vetor deslocamentos: {desloc.flatten()}')

    # Reações
    reacoes = K @ desloc  # Vetor reações
    # print(f'Vetor reações: {reacoes.flatten()}')

    # Esforços nos elementos
    fn = np.zeros(n_el)
    ten = np.zeros(n_el)
    for el in range(n_el):
        # cálculo do comprimento do elemento "el"
        no1 = conec[el][-2]-1
        no2 = conec[el][-1]-1

        L = np.sqrt((x[no2] - x[no1])**2 + (y[no2] - y[no1])**2)

        # Propriedades
        s = prop_group[conec[el][1]-1][0]
        m = prop_group[conec[el][1]-1][1]
        A = secoes[s-1][0]
        E = material[m-1][0]

        # Cossenos diretores a partir das coordenadas dos nós do elemento
        cs = (x[no2] - x[no1])/L    # cosseno
        sn = (y[no2] - y[no1])/L    # seno

        # pega os valores dos deslocamentos dos nós do elemento "el"
        u1 = desloc[(no1+1)*2-2]
        u2 = desloc[(no2+1)*2-2]
        v1 = desloc[(no1+1)*2-1]
        v2 = desloc[(no2+1)*2-1]

        # constante de rigidez do elemento "el"
        k = E*A/L

        # força e tensão atuante no elemento "el"
        # cálculo da força normal no elemento
        fn[el] = k*(-(u1-u2)*cs - (v1-v2)*sn)
        # cálculo da tensão normal no elemento
        ten[el] = fn[el]/A
    return desloc, fn, ten, reacoes

In [12]:
import pandas as pd
import numpy as np

def save_truss_info(data, secoes, material, group, conec, x, y, GDL_rest, forcas, no_critico, pp, gravidade, gama_d, n):
    filename = 'output_' + str(n) + '.xlsx'
    with pd.ExcelWriter(filename) as writer:
        data.to_excel(writer, sheet_name="analysis")

        # Salva os materiais e seção na database
        df_sec = pd.DataFrame(secoes, index=list(range(1, secoes.shape[0]+1)), columns = ["A", "b", "t", "Ix", "Iy", "rx", "ry", "rz_min", "wdt", "J", "W", "x", "s4g"])
        df_sec.to_excel(writer, sheet_name="sections")

        df_mat = pd.DataFrame(material, index=list(range(1, material.shape[0]+1)), columns = ["E", "fy_k", "density"])
        df_mat.to_excel(writer, sheet_name="materials")

        # Salva os grupos
        df_gru = pd.DataFrame(group, index=list(range(1, group.shape[0]+1)), columns=np.zeros((group.shape[1])))
        df_gru.to_excel(writer, sheet_name="groups")
        
        # Salva as coordenadas e conectividades
        coord = np.array([x,y])
        df_coo = pd.DataFrame(coord, index=['x', 'y'], columns=list(range(1, x.shape[0]+1)))
        df_coo.to_excel(writer, sheet_name="coord")
        
        df_con = pd.DataFrame(conec[:,-2:], index=list(range(1, conec.shape[0]+1)), columns=['node 1', 'node 2'])
        df_con.to_excel(writer, sheet_name="elements")
        
        # Salva os GDL restringidos
        df_res = pd.DataFrame(GDL_rest, index=list(range(1, GDL_rest.shape[0]+1)), columns=['node', 'rest x', 'rest y'])
        df_res.to_excel(writer, sheet_name="GDL_rest")
        
        # Salva as forças
        df_for = pd.DataFrame(forcas, index=list(range(1, forcas.shape[0]+1)), columns=['node', 'Fx', 'Fy'])
        df_for.to_excel(writer, sheet_name="ext forces")
        
        # Salva informações
        df_inf = pd.DataFrame([pp, gravidade, gama_d, no_critico], index=['self weight coef', 'gravity', 'fy_k reduction', 'observed node'])
        df_inf.to_excel(writer, sheet_name="info")

In [13]:
import numpy as np

def self_weight(x, y, conec, prop_group, secoes, material, pp, gravidade, forcas, GDL_rest):
    """
    Adiciona o peso próprio, se houver.

    Parameters:
            x (np.float array of shape (n_nos,)):
                Coordinates x of nodes
            y (np.float array of shape (n_nos,)):
                Coordinates y of nodes
            conec (np.int array of shape (n_el, 4)): 
                [elemento, grupo,  nó_1,  nó_2]
            prop_group (np.int array of shape (n_group, 2)):
                Seção e material adotados em cada grupo
            secoes (np.float array of shape (n_sec, 13)):
                [Área, b, t, Ix, Iy, rx, ry, rz_min, wdt, J, W, x, s4g]
            material (np.float array of shape (n_mat, 3)):
                [Young_modulus, fy_k, density]
            pp (np.float):
                self weight multiplier
            gravidade (np.float):
                gravity in m/s**2
            forcas (np.float array of shape (n_forcas, 3)):
                [nó, Fx, Fy]
            GDL_rest (np.int array of shape (n_rest, 3)):
                [nó, rest_x, rest_y]
            
    Returns:
            forcas (np.float array of shape (n_forcas, 3)):
                [nó, Fx, Fy]
    """
    
    no = np.arange(1, len(x)+1)  # numeração dos nós
    n_nos = no[-1]  # número de nós
    n_el = conec.shape[0]
    n_forcas = forcas.shape[0]
    n_rest = GDL_rest.shape[0]  # número de nós restringidos

    if pp > 0:
        peso_proprio = np.empty([1, 3])
        for el in range(n_el):
            s = prop_group[conec[el][1]-1][0]
            m = prop_group[conec[el][1]-1][1]
            A = secoes[s-1][0]
            density = material[m-1][2]

            no1 = conec[el][-2]-1
            no2 = conec[el][-1]-1

            L = np.sqrt((x[no2] - x[no1])**2 + (y[no2] - y[no1])**2)

            massa = L*A*density
            peso = massa*gravidade*pp

            if conec[el][3] in GDL_rest[:,0] or conec[el][4] in GDL_rest[:,0]:
                for i in range(n_rest):
                    if conec[el][3] == GDL_rest[i][0] and GDL_rest[i][2] == 1:
                        peso_proprio = np.concatenate(
                            (peso_proprio, np.array([[conec[el][4], 0, -peso]])), axis=0)
                        break
                    elif conec[el][4] == GDL_rest[i][0] and GDL_rest[i][2] == 1:
                        peso_proprio = np.concatenate(
                            (peso_proprio, np.array([[conec[el][3], 0, -peso]])), axis=0)
                        break
            else:
                peso_proprio = np.concatenate((peso_proprio,
                                            np.array([[conec[el][3], 0, -peso/2],
                                                        [conec[el][4], 0, -peso/2]])),
                                            axis=0)
        peso_proprio = np.delete(peso_proprio, 0, 0)
        
        for p in range(peso_proprio.shape[0]):
            node_p = peso_proprio[p][0]
            
            if node_p in forcas[:,0]:
                for f in range(n_forcas):
                    node_f = forcas[f][0]
                    
                    if node_p == node_f:
                        forcas[f][2] = forcas[f][2] + peso_proprio[p][2]
                        min_node_f = n_nos+1
                        break
            else:
                forcas = np.concatenate((forcas, [peso_proprio[p]]), axis=0)
            n_forcas = forcas.shape[0]

    return forcas

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
from sklearn import preprocessing


def plot_truss(F, sigma, forcas, x, y, conec, desloc, scale, desloc_adm_x, desloc_adm_y, prop_group, material, gama_d):
    """
    PLOTA A ÚLTIMA ESTRUTURA ANALISADA
    """
    fn = F
    ten = sigma
    no = np.arange(1, len(x)+1)  # numeração dos nós
    n_nos = no[-1]  # número de nós
    n_forcas = forcas.shape[0] # número de forças
    n_el = conec.shape[0] # número de elementos

    print(f'Vetor de esforços: {fn}')
    print(f'Vetor de tensões:  {ten}')
    print(f'Esforço axial no elemento 2: {fn[1]} \n')

    print('===========================================')
    print('NÓ        UX                             UY')
    print('-------------------------------------------')
    x_deformed = np.copy(x)
    y_deformed = np.copy(y)
    falha = False
    for i in range(n_nos):
        x_deformed[i] = x[i] + desloc[2*(i+1)-2]*scale
        y_deformed[i] = y[i] + desloc[2*(i+1)-1]*scale
        print(
            f'Nó {i+1:2.0f} --> Deslocamento x: {float(desloc[2*(i+1)-2]):10.5f} m;  Deslocamento y: {float(desloc[2*(i+1)-1]):10.5f} m')

        desloc_taxa = np.array(
            [desloc[2*(i+1)-2]/desloc_adm_x, desloc[2*(i+1)-1]/desloc_adm_y])
        if np.abs(desloc_taxa[0][0]) > 1:
            falha = True
            print(
                f'NÓ {i+1} ULTRAPASSOU O DESLOCAMENTO MÁXIMO ADMISSÍVEL EM {np.abs(desloc_taxa[0][0]-1)*100:4.1f}%')

        if np.abs(desloc_taxa[1][0]) > 1:
            falha = True
            print(
                f'NÓ {i+1} ULTRAPASSOU O DESLOCAMENTO MÁXIMO ADMISSÍVEL EM {np.abs(desloc_taxa[1][0]-1)*100:4.1f}%')

    print('\n')

    print('-----------------------------------------------------------')
    print('ELEMENTO        FORÇA AXIAL                   TENSÃO NORMAL')
    print('-----------------------------------------------------------')
    for el in range(n_el):
        print(
            f'Elemento {i+1:2.0f} --> Força axial: {float(fn[i]):15.5f}; Tensão normal: {float(ten[i]):10.5f}')
        m = prop_group[conec[el][1]-1][1]
        fy_taxa = np.abs(ten[i])/(material[m-1][1]/gama_d)
        if fy_taxa > 1:
            falha = True
            print(
                f'ELEMENTO {conec[i][0]} FALHOU. ULTRAPASSOU A TENSÃO DE ESCOAMENTO EM {(fy_taxa-1)*100:4.1f}%')
    print('===========================================================')
    print('\n')

    if falha:
        print('A ESTRUTURA NÃO ESTÁ SEGURA!')

    # Plots
    plt.style.use('_mpl-gallery')
    font = {'family': 'sans-serif',
            'weight': 'normal',
            'size': 20}

    plt.rc('font', **font)

    fig, (ax1, ax2) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [16, 1]})

    cmap = mpl.cm.bwr
    norm = mpl.colors.Normalize(vmin=min(fn), vmax=max(fn))
    cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap,
                                    norm=norm,
                                    orientation='vertical')

    xmin = min(x)
    xmax = max(x)

    ymin = min(y)
    ymax = max(y)

    # Plota treliça
    find_nearest = lambda array, value: (np.abs(array - value)).argmin()
    for i in range(n_el):
        x_el = np.array(x[conec[i][-2]-1], x[conec[i][-1]-1])
        y_el = np.array(y[conec[i][-2]-1], y[conec[i][-1]-1])

        x_el_deformed = np.array(
            [x_deformed[conec[i][-2]-1], x_deformed[conec[i][-1]-1]])
        y_el_deformed = np.array(
            [y_deformed[conec[i][-2]-1], y_deformed[conec[i][-1]-1]])

        idx = find_nearest(np.array(cb1._values), fn[i])

        ax1.plot(x_el_deformed, y_el_deformed, linewidth=3, c=cm.bwr(idx))

    # Plota forças (representadas por linhas tracejadas pretas)
    normalized = preprocessing.normalize([forcas[:,1:].flatten()])

    no1 = conec[-1][-2]-1
    no2 = conec[-1][-1]-1
    L = np.sqrt((x[no2] - x[no1])**2 + (y[no2] - y[no1])**2)

    for i in range(n_forcas):
        fx_i = x_deformed[int(forcas[i][0]-1)]
        fy_i = y_deformed[int(forcas[i][0]-1)]

        fx_size = normalized[0][i*2]*L/7
        fy_size = normalized[0][i*2+1]*L/7

        ax1.plot(np.array([fx_i, fx_i+fx_size]),
                np.array([fy_i, fy_i]),
                linewidth=2, c='k', linestyle='--')
        
        ax1.plot(np.array([fx_i, fx_i]),
                np.array([fy_i, fy_i+fy_size]),
                linewidth=2, c='k', linestyle='--')
    
    # Plota texto nos elementos
    for el in range(n_el):
        # Comprimento "L" do elemento "el"
        no1 = conec[el][-2]-1
        no2 = conec[el][-1]-1

        x_el_deformed = np.array(
            [(x_deformed[no1]+x[no1])/2, (x_deformed[no2]+x[no2])/2])
        y_el_deformed = np.array(
            [(y_deformed[no1]+y[no1])/2, (y_deformed[no2]+y[no2])/2])

        x_el = np.linspace(x_el_deformed[0], x_el_deformed[1], 8)
        y_el = np.linspace(y_el_deformed[0], y_el_deformed[1], 8)

        ax1.text(x_el[int(len(x_el)/2)], y_el[int(len(y_el)/2)], str(el), color='c')

    # Plota texto nos nós
    for no in range(n_nos):
        ax1.text(x_deformed[no], y_deformed[no], str(no), color='m')


    # Ajusta algumas configurações dos plots
    plt.subplots_adjust(bottom=0.1, right=0.9, left=0.1, top=0.9)

    ax1.set_xlabel('x (m)')
    ax1.set_ylabel('y (m)')
    ax2.set_xlabel('Esforço axial (N)')

    plt.grid(color='0.5', linestyle=':', linewidth=1)
    ax1.set_title(f'Treliça na configuração deformada: Escala {scale}')

    plt.show()


In [15]:
import pandas as pd

def set_up_datatable(group):
    """Set up data table header."""

    # Monta a tabela de dados
    string_sec = []
    string_mat = []
    string_sigma = []
    # string_taxa_fy = []
    string_total = []
    for g in range(group.shape[0]):
        string_sec.append("Section G"+str(g))
        string_mat.append("Material G"+str(g))
        string_sigma.append("Tension G"+str(g))
        # string_taxa_fy.append("sigma/sigma_adm G"+str(g)) # > 1 indica falha
    string_total.extend(string_sec)
    string_total.extend(string_mat)
    string_total.extend(["Fx", "Fy", "desloc_x", "desloc_y"])
    string_total.extend(string_sigma)
    # string_total.extend(string_taxa_fy)
    data = pd.DataFrame(columns=(string_total))
    
    return data

In [18]:
import numpy as np
import pandas as pd
import random
import time
from datetime import timedelta

def trelica(secao_grupo, plotagem=False, salva_excel=False):

    """
    SET PARAMETERS
    """
    print(secao_grupo)
    # Parâmetros
    var_sec = False
    var_mat = False
    var_Fx = False
    var_Fy = False
    n = 1 # number of times to run

    # Montagem da estrutura
    # Nós
    x = np.array([0, 0, 1, 1, 2], dtype='float64')
    y = np.array([0, 1, 0, 1, 0], dtype='float64')

    # Escala da deformação (só para visualização do plot)
    scale = 5

    # Deslocamento admissível
    desloc_adm_x = 0.005  # m
    desloc_adm_y = 0.01  # m
    no_critico = 5 # número do nó onde o desloc adm será observado

    # Materiais
    Young_modulus = [  # módulo de elasticidade
        200e9,
    ]

    fyk = [  # tensão de escoamento
        344.7e6,  # ASTM A572 G50
        413.6e6,  # ASTM A572 G60
    ]

    gama_d = 1.0 # fator de redução de fyk

    density = [
        7836.41
    ]

    # Multiplicador do peso próprio
    pp = 0  # 0 para não considerar o peso próprio da estrutura
    gravidade = 9.80665  # m/s**2

    material = np.array([
        [Young_modulus[0], fyk[0], density[0]],
        [Young_modulus[0], fyk[1], density[0]],
    ], dtype='float64')

    # Seções [número da seção,  área]
    secoes = np.array([
    #    Área      b      t      Ix        Iy        rx        ry        rz mín    wdt    J         W         x         s4g
        [2.35E-04, 0.04,  0.003, 3.45E-08, 3.45E-08, 1.21E-02, 1.21E-02, 7.80E-03, 10.33, 6.93E-10, 1.24E-06, 1.11E-02, 0], # 1 L40x40x3
        [3.08E-04, 0.04,  0.004, 4.61E-08, 4.61E-08, 1.21E-02, 1.21E-02, 7.80E-03, 7.5,   1.56E-09, 1.55E-06, 1.15E-02, 0], # 2 L40x40x4
        [3.79E-04, 0.04,  0.005, 5.43E-08, 5.43E-08, 1.20E-02, 1.20E-02, 7.70E-03, 5.8,   2.97E-09, 1.97E-06, 1.18E-02, 0], # 3 L40x40x5
        [2.66E-04, 0.045, 0.003, 4.93E-08, 4.93E-08, 1.36E-02, 1.36E-02, 8.80E-03, 11.67, 7.83E-10, 1.58E-06, 1.23E-02, 0], # 4 L45x45x3
        [3.49E-04, 0.045, 0.004, 6.67E-08, 6.67E-08, 1.36E-02, 1.36E-02, 8.70E-03, 8.5,   1.77E-09, 2.07E-06, 1.28E-02, 0], # 5 L45x45x4
        [4.30E-04, 0.045, 0.005, 7.84E-08, 7.84E-08, 1.35E-02, 1.35E-02, 8.70E-03, 6.6,   3.54E-09, 2.43E-06, 1.40E-02, 0], # 6 L45x45x5
        [2.96E-04, 0.05,  0.003, 7.15E-08, 7.15E-08, 1.52E-02, 1.52E-02, 9.90E-03, 13.33, 8.53E-10, 1.96E-06, 1.35E-02, 0], # 7 L50x50x3
        [3.89E-04, 0.05,  0.004, 9.26E-08, 9.26E-08, 1.52E-02, 1.52E-02, 9.80E-03, 9.75,  1.99E-09, 2.57E-06, 1.40E-02, 0], # 8 L50x50x4
        [4.80E-04, 0.05,  0.005, 1.10E-07, 1.10E-07, 1.51E-02, 1.51E-02, 9.70E-03, 7.6,   3.96E-09, 3.05E-06, 1.42E-02, 0], # 9 L50x50x5
        [4.71E-04, 0.06,  0.004, 1.63E-07, 1.63E-07, 1.83E-02, 1.83E-02, 1.18E-02, 12,    2.41E-09, 3.75E-06, 1.65E-02, 0], # 10 L60x60x4
        [5.82E-04, 0.06,  0.005, 1.99E-07, 1.99E-07, 1.82E-02, 1.82E-02, 1.17E-02, 9.4,   4.64E-09, 4.45E-06, 1.64E-02, 0], # 11 L60x60x5
        [5.13E-04, 0.065, 0.004, 2.09E-07, 2.09E-07, 1.98E-02, 1.98E-02, 1.28E-02, 13,    2.63E-09, 4.42E-06, 1.77E-02, 3.84E-02], # 12 L65x65x4
        [6.31E-04, 0.065, 0.005, 2.47E-07, 2.47E-07, 1.98E-02, 1.98E-02, 1.28E-02, 10.2,  5.06E-09, 5.20E-06, 1.77E-02, 3.84E-02], # 13 L65x65x5
    ], dtype='float64')

    # Elementos (conectividades) [elemento,   grupo,   nó_1,    nó_2]
    conec = np.array([
        [1, 1, 1, 3],
        [2, 2, 3, 4],
        [3, 3, 2, 4],
        [4, 4, 1, 4],
        [5, 5, 2, 3],
        [6, 6, 3, 5],
        [7, 7, 4, 5],
    ], dtype='int32')

    # Seção e material do grupo
    prop_group =  np.array([ # [seção, material]
        [13, 1],
        [13, 1],
        [13, 1],
        [13, 1],
        [13, 1],
        [13, 1],
        [13, 1],
    ], dtype='int32')

    for idx, s in enumerate(secao_grupo):
        prop_group[idx][0] = s

    # Carregamentos [nó,   intensidade_x,  intensidade_y]
    forcas = np.array([
        [5, 0, -1e5]
    ])

    # Apoios
    GDL_rest = np.array([  # [nó, restringido_x, restringido_y] (1 para restringido, e 0 para livre)
        [1, 1, 1],
        [2, 1, 1],
    ], dtype='int32')


    """
    VARIA AS PROPRIEDADES/FORÇAS
    E REALIZA A ANÁLISE
    """
    n_el = conec.shape[0]
    # set group variable
    n_groups = np.unique(conec[:, 1]).shape[0]
    group = np.zeros((n_groups, n_el), dtype='int32')
    selected_mat = np.zeros((n_groups), dtype='int32')
    selected_sec = np.zeros((n_groups), dtype='int32')
    for el in range(n_el):
        idx = conec[el][1]-1
        g = group[idx]
        group[idx][np.count_nonzero(g)] = el+1
        
        selected_mat[idx] = prop_group[idx][1]-1
        selected_sec[idx] = prop_group[idx][0]-1

    # Set up data table
    data = set_up_datatable(group)

    ### Varia as propriedades, forças, etc
    for _ in range(n):
        # start = time.time()
        if var_Fx:
            fx = 0  # em Newtons
            forcas[0][1] = fx
        else:
            fx = forcas[0][1]
            
        if var_Fy:
            fy = random.randint(-10**6, 10**6)  # em Newtons
            forcas[0][2] = fy
        else:
            fy = forcas[0][2]

        if var_mat:
            E_group, E, selected_mat = vary_groups(group, n_el, material, 0, [])
            fy_k_group, fy_k, selected_mat = vary_groups(group, n_el, material, 1, np.array([]))
        else:
            E_group, E, selected_mat = vary_groups(group, n_el, material, 0, selected_mat)
            fy_k_group, fy_k, selected_mat = vary_groups(group, n_el, material, 1, selected_mat)
            
        if var_sec:
            area_group, area, selected_sec = vary_groups(group, n_el, secoes, 0, np.array([]))
        else:
            area_group, area, selected_sec = vary_groups(group, n_el, secoes, 0, selected_sec)
        
        # Realiza análise
        prop_group[:,0] = selected_sec+1
        prop_group[:,1] = selected_mat+1
        forcas = self_weight(x, y, conec, prop_group, secoes, material, pp, gravidade, forcas, GDL_rest)
        desloc, F, sigma, reacoes = FEM(x, y, conec, prop_group, secoes, material, forcas, GDL_rest)
        
        # Agrupa resultados (se houverem diferentes grupos)
        F_group = group_results(F, group)
        sigma_group = group_results(sigma, group)

        # Deslocamento do nó mais crítico
        dx = desloc[2*(no_critico)-2]
        dy = desloc[2*(no_critico)-1]

        # Guarda na tabela 'data'
       # data.loc[len(data)] = np.concatenate((area_group, E_group, np.array([fx]), np.array([fy]), dx, dy,
     #                                       sigma_group))
        
        # Estima o tempo que vai demorar todas as 'n' iterações
        #end = time.time()
        # porcentagem = (_/n)*100
        # if porcentagem % 20 == 0:
        #     duration = end - start
        #     total_time = duration * n
        #     elapsed_time = duration * _
        #     remaining_time = total_time - elapsed_time
            
        #     print(f'Elapsed time (hh:mm:ss) {timedelta(seconds=elapsed_time)} --- '  
        #         f'Reamining time (hh:mm:ss) {timedelta(seconds=remaining_time)} --- '
        #         f'Total time (hh:mm:ss) {timedelta(seconds=total_time)}')


    """
    SALVA OS DADOS EM EXCEL
    """
    #if salva_excel:
       # save_truss_info(data, secoes, material, group, conec, x, y, GDL_rest, forcas, no_critico, pp, gravidade, gama_d, n)
        

    """
    PLOTA A ÚLTIMA ESTRUTURA ANALISADA
    """
    #if plotagem:
     #   plot_truss(F, sigma, forcas, x, y, conec, desloc, scale, desloc_adm_x, desloc_adm_y, prop_group, material, gama_d)

    """
    RETORNA O CUSTO DA ESTRUTURA
    """
    cost = 0
    for el in range(n_el):
        # Comprimento "L" do elemento "el"
        no1 = conec[el][-2]-1
        no2 = conec[el][-1]-1

        L = np.sqrt((x[no2] - x[no1])**2 + (y[no2] - y[no1])**2)

        # Propriedades
        s = prop_group[conec[el][1]-1][0]
        A = secoes[s-1][0]

        V = L*A # Volume do elemento

        cost = cost + V

    taxa_dy = abs(dy/desloc_adm_y)
    if taxa_dy > 1:
        cost = cost + taxa_dy*10**8

    return cost

# if __name__ == "__main__":
#print(trelica([1,1,13,1,1,1,1], True, False))

In [19]:
from scipy.optimize import shgo

bounds = (1,13),(1,13),(1,13),(1,13),(1,13),(1,13),(1,13)

result = shgo(trelica, bounds)

[1. 1. 1. 1. 1. 1. 1.]
[13. 13. 13. 13. 13. 13. 13.]
[13.  1.  1.  1.  1.  1.  1.]
[13. 13.  1.  1.  1.  1.  1.]
[13. 13. 13.  1.  1.  1.  1.]
[13. 13. 13. 13.  1.  1.  1.]
[13. 13. 13. 13. 13.  1.  1.]
[13. 13. 13. 13. 13. 13.  1.]
[13. 13. 13. 13. 13.  1. 13.]
[13. 13. 13. 13.  1. 13.  1.]
[13. 13. 13. 13.  1. 13. 13.]
[13. 13. 13. 13.  1.  1. 13.]
[13. 13. 13.  1. 13.  1.  1.]
[13. 13. 13.  1. 13. 13.  1.]
[13. 13. 13.  1. 13. 13. 13.]
[13. 13. 13.  1. 13.  1. 13.]
[13. 13. 13.  1.  1. 13.  1.]
[13. 13. 13.  1.  1. 13. 13.]
[13. 13. 13.  1.  1.  1. 13.]
[13. 13.  1. 13.  1.  1.  1.]
[13. 13.  1. 13. 13.  1.  1.]
[13. 13.  1. 13. 13. 13.  1.]
[13. 13.  1. 13. 13. 13. 13.]
[13. 13.  1. 13. 13.  1. 13.]
[13. 13.  1. 13.  1. 13.  1.]
[13. 13.  1. 13.  1. 13. 13.]
[13. 13.  1. 13.  1.  1. 13.]
[13. 13.  1.  1. 13.  1.  1.]
[13. 13.  1.  1. 13. 13.  1.]
[13. 13.  1.  1. 13. 13. 13.]
[13. 13.  1.  1. 13.  1. 13.]
[13. 13.  1.  1.  1. 13.  1.]
[13. 13.  1.  1.  1. 13. 13.]
[13. 13.  1.  1. 